## Anydice: a pool where stress dice cancel out action dice of equal or lower value and return the number and value of action dice remaining in the pool?

[StackExchange question.](https://rpg.stackexchange.com/questions/198555/anydice-a-pool-where-stress-dice-cancel-out-action-dice-of-equal-or-lower-value)

1. Roll a pool of action dice and a pool of stress dice. In this question all dice are d6s.
2. Sort the results of each pool from highest to lowest.
3. Starting from the highest stress die, each stress die cancels out the highest action die of equal or lesser value.
  (If there is no such action die, that stress die has no effect.)
5. The result is the multiset of remaining action dice.

In [1]:
import piplite
await piplite.install("icepool")

import icepool
from icepool import d6, MultisetEvaluator

class AllActionStress(MultisetEvaluator):
    def next_state(self, state, outcome, action, stress):
        # This is called for each possible outcome 6, 5, 4, 3, 2, 1
        # with the number of action and stress dice that rolled that number.
        # This method uses this information to update a "running total".
        
        # action_set: The set of surviving action dice.
        # leftover_stress: The number of surviving stress dice that rolled this number or higher.
        action_set, leftover_stress = state or ((), 0)
        leftover_stress += stress
        if leftover_stress >= action:
            leftover_stress -= action
        else:
            action -= leftover_stress
            action_set += (outcome,) * action
            leftover_stress = 0
        
        return action_set, leftover_stress
    
    def final_outcome(self, final_state, *pools):
        # Just return the action set; any remaining stress is ignored.
        return final_state[0]
    
    def order(self):
        # See outcomes in descending order.
        return -1

all_action_stress = AllActionStress()
result = all_action_stress.evaluate(d6.pool(5), d6.pool(3))

# The number of surviving action dice.
# If this is all you are interested in, it would be more efficient to only count the number
# of surviving action dice in AllActionStress, rather than computing the entire surviving
# action sets and counting the length at the end as we do here.
print(result.map(len))

Denominator: 1679616

| Outcome |  Weight | Probability |
|--------:|--------:|------------:|
|       2 | 1097586 |  65.347437% |
|       3 |  432960 |  25.777321% |
|       4 |  132915 |   7.913416% |
|       5 |   16155 |   0.961827% |



In [2]:
# We can print the entire action sets, but this is a lot of rows especially as the action pool gets larger.
print(result)

Denominator: 1679616

|         Outcome | Weight | Probability |
|----------------:|-------:|------------:|
|          (1, 1) |  97591 |   5.810316% |
|          (2, 1) |  97630 |   5.812638% |
|          (2, 2) |  48810 |   2.906021% |
|       (2, 2, 2) |   5710 |   0.339959% |
|    (2, 2, 2, 2) |    170 |   0.010121% |
| (2, 2, 2, 2, 2) |      1 |   0.000060% |
|          (3, 1) |  59200 |   3.524615% |
|          (3, 2) |  48495 |   2.887267% |
|       (3, 2, 2) |  10185 |   0.606389% |
|    (3, 2, 2, 2) |    515 |   0.030662% |
| (3, 2, 2, 2, 2) |      5 |   0.000298% |
|          (3, 3) |  34734 |   2.067973% |
|       (3, 3, 2) |   9210 |   0.548340% |
|    (3, 3, 2, 2) |    720 |   0.042867% |
| (3, 3, 2, 2, 2) |     10 |   0.000595% |
|       (3, 3, 3) |   7076 |   0.421287% |
|    (3, 3, 3, 2) |    470 |   0.027983% |
| (3, 3, 3, 2, 2) |     10 |   0.000595% |
|    (3, 3, 3, 3) |    483 |   0.028757% |
| (3, 3, 3, 3, 2) |      5 |   0.000298% |
| (3, 3, 3, 3, 3) |      8 |   0